In [1]:
import glob
import ipywidgets
import ipyvolume
import cv2
import numpy as np
import os
import time

In [2]:
experiment = 'MWCNN_DeNoising'
data_dir = '/home/phacou/MWCNNv2/MWCNN_code/experiment/'+experiment+'/'
sample = 'ucsdLEDsm'
noise = '1k' # other values you can try: '15' '20' '25' '30' '40'

In [3]:
model_addr = data_dir+sample+'_results/'

In [4]:
case_addr = model_addr + noise
os.path.join(case_addr,'GT/*HR.png')

'/home/phacou/MWCNNv2/MWCNN_code/experiment/MWCNN_DeNoising/ucsdLEDsm_results/1k/GT/*HR.png'

In [ ]:
case_addr = model_addr + noise
HRpngs = sorted(glob.glob(os.path.join(case_addr,'GT/*HR.png')))
LRpngs = sorted(glob.glob(os.path.join(case_addr,'noisy/*LR.png')))
SRpngs = sorted(glob.glob(os.path.join(case_addr,'model/*SR.png')))
#hrdata = []
lrdata = []
srdata = []
print(case_addr)
for (hr,lr,sr) in zip(HRpngs,LRpngs,SRpngs):
    #hrf = cv2.imread(hr, 0)
    lrf = cv2.imread(lr, 0)
    srf = cv2.imread(sr, 0)
    #if (hrf.max() > hrf.min()):
    #    hrf = (hrf - hrf.min())/ (hrf.max() - hrf.min())
    lrf = (lrf - lrf.min())/ (lrf.max() - lrf.min())
    srf = (srf - srf.min())/ (srf.max() - srf.min())
    #print(lr.replace('noisy/','noisy_norm/'),sr.replace('model/','model_norm/'))
    #cv2.imwrite(lr.replace('noisy/','noisy_norm/'),lrf*255.)
    #cv2.imwrite(sr.replace('model/','model_norm/'),srf*255.)
    #hrdata.append(hrf)
    srdata.append(srf)
    lrdata.append(lrf)
#hrdata = np.array(hrdata)
lrdata = np.array(lrdata)
srdata = np.array(srdata)

/home/phacou/MWCNNv2/MWCNN_code/experiment/MWCNN_DeNoising/ucsdLEDsm_results/1k


In [ ]:
case_addr = model_addr + noise
HRpngs = sorted(glob.glob(os.path.join(case_addr,'GT/*HR.png')))
hrdata = []
print(case_addr)
for hr in HRpngs:
    hrf = cv2.imread(hr, 0)
    hrdata.append(hrf)
hrdata = np.array(hrdata)

In [ ]:
np.save('lr_'+sample+'_vol_'+noise+'.npy',lrdata)
np.save('sr_'+sample+'_vol_'+noise+'.npy',srdata)
np.save('gt_'+sample+'_vol_'+noise+'.npy',hrdata)

In [ ]:
ipyvolume.figure()
ipyvolume.volshow(lrdata[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
ipyvolume.show()

In [ ]:
ipyvolume.view(azimuth=90, elevation=5, distance=3)

In [ ]:
ipyvolume.pylab.save(model_addr + noise + '_lr_3dfig.html')
ipyvolume.pylab.savefig(model_addr +noise+'_lr_3dfig.png',1280,720)

In [ ]:
ipyvolume.figure()
ipyvolume.volshow(srdata[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
ipyvolume.show()

In [ ]:
ipyvolume.view(azimuth=90, elevation=5, distance=3)

## Convert data into MATLAB

In [ ]:
import scipy.io as sio

In [ ]:
noise = '20'

In [ ]:
model_addr = '/home/phacou/MWCNNv2/MWCNN_code/experiment/'+experiment+'/'+sample+'_results/'
case_addr = model_addr + noise
HRpngs = sorted(glob.glob(case_addr+'/GT/*HR.png'))
LRpngs = sorted(glob.glob(case_addr+'/noisy/*LR.png'))
SRpngs = sorted(glob.glob(case_addr+'/model/*SR.png'))
hrdata = []
lrdata = []
srdata = []
for (hr,lr,sr) in zip(HRpngs,LRpngs,SRpngs):
    #hrf = cv2.imread(hr, 0)
    lrf = cv2.imread(lr, 0)
    srf = cv2.imread(sr, 0)
    #if (hrf.max() > hrf.min()):
    #    hrf = (hrf - hrf.min())/ (hrf.max() - hrf.min())
    lrf = (lrf - lrf.min())/ (lrf.max() - lrf.min())
    srf = (srf - srf.min())/ (srf.max() - srf.min())
    hrdata.append(hrf)
    srdata.append(srf)
    lrdata.append(lrf)
hrdata = np.array(hrdata)
lrdata = np.array(lrdata)
srdata = np.array(srdata)

In [ ]:
#sio.savemat('U900_15_HR.mat',{'vect':hrdata})
sio.savemat('U900_'+noise+'_LR.mat',{'vect':lrdata})
sio.savemat('U900_'+noise+'_SR.mat',{'vect':srdata})

## Plot averaged slided results

## Generate the output results by averaging slided outputs

generate the volume data of the model output

In [ ]:
width = 390
height = 90
slide = 10 # number of pixels slided
nslide = int((width - height)/slide) + 1 # number of slided frames within each input frame (390 - 90)/10 + 1
sample = "ucsdLEDsm" # "ucsd900all"
folder_prefix = 'experiment/MWCNN_DeNoising/'+sample+'_results'
noises = list(map(lambda x: x.split('/')[-1], glob.glob(folder_prefix+'/*')))
for n in noises:
    data_folder = os.path.join(folder_prefix,noises[0]) # folder_prefix+n+'/'
    print('Data folder: {}'.format(data_folder))
    pngs = glob.glob(os.path.join(data_folder,'model/*SR.png'))
    nframes = int(len(pngs)/nslide)
    inds = list(map(lambda x: int(x.split('/')[-1].split('_')[2]),pngs))
    png_files = [files for _,files in sorted(zip(inds,pngs))]
    #print(os.path.join(data_folders,'*.png'),pngs)
    full_img = np.zeros((height,width,nframes),dtype=float)
    full_img_int = np.zeros((height,width,nframes),dtype=int)
    for nimg, pngf in enumerate(png_files):
        img = np.array(cv2.resize(cv2.imread(pngf),(height,height)))[:,:,0]
        #img = (img - img.min()) / (img.max() - img.min())
        nframe = int(nimg / nslide)
        ns = nimg - nframe * nslide
        full_img[:,(ns*slide):(ns*slide+height),nframe] = full_img[:,(ns*slide):(ns*slide+height),nframe] + img
        full_img_int[:,(ns*slide):(ns*slide+height),nframe] = full_img_int[:,(ns*slide):(ns*slide+height),nframe] + np.ones((height,height),dtype=int)
        #print(pngf, nframe, ns, img.shape)
    avg_img = full_img / full_img_int
    np.save("result_vol_npys/output_"+sample+"_vol_"+n+".npy", avg_img)

generate the model input volume

## For comparison generate the input data, also target data

In [ ]:
input_folder = '../data/ucsdLEDsm/' #'../data/crop_all_letters/'
noises = ['4k']
print('Input folder: {}'.format(input_folder))
target_jpgs = sorted(glob.glob(os.path.join(input_folder,'*_'+noises[0]+'_*target.jpg')))
target_vol = []
for tar in target_jpgs:
    target_vol.append(cv2.imread(tar))
target_vol = np.transpose(target_vol,[1,2,0,3])[:,:,:,0]
np.save('result_vol_npys/target_'+sample+'_vol.npy',target_vol)
for noise in noises:
    input_jpgs = sorted(glob.glob(os.path.join(input_folder,'*_'+noise+'_*input.jpg')))
    input_vol = []
    for inp,tar in zip(input_jpgs,target_jpgs):
        ind = int(inp.split('/')[-1].split('_')[0]) - 1
        input_vol.append(cv2.imread(inp))
        #print(ind,inp,tar)
    input_vol = np.transpose(input_vol,[1,2,0,3])[:,:,:,0]
    np.save('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy',input_vol)

In [ ]:
input_folder = '../data/crop_all_letters/'
print('Input folder: {}'.format(input_folder))
target_jpgs = sorted(glob.glob(os.path.join(input_folder,'*_'+noises[0]+'_*target.jpg')))
target_vol = []
for tar in target_jpgs:
    target_vol.append(cv2.imread(tar))
target_vol = np.transpose(target_vol,[1,2,0,3])[:,:,:,0]
np.save('result_vol_npys/target_ucsd_vol.npy',target_vol)
for noise in noises:
    input_jpgs = sorted(glob.glob(os.path.join(input_folder,'*_'+noise+'_*input.jpg')))
    input_vol = []
    for inp,tar in zip(input_jpgs,target_jpgs):
        ind = int(inp.split('/')[-1].split('_')[0]) - 1
        input_vol.append(cv2.imread(inp))
        #print(ind,inp,tar)
    input_vol = np.transpose(input_vol,[1,2,0,3])[:,:,:,0]
    np.save('result_vol_npys/input_ucsd_vol_'+noise+'.npy',input_vol)

In [ ]:
for noise in ['10','15','20','25','30','40']:
    data = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
    data = np.flip(data,0)
    data = np.flip(data,2)
    data = np.transpose(data, (2,0,1))
    np.save('result_vol_npys/output_'+sample+'flipped_vol_'+noise+'.npy',data)
    data = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
    data = np.flip(data,0)
    data = np.flip(data,2)
    data = np.transpose(data, (2,0,1))
    np.save('result_vol_npys/input_'+sample+'flipped_vol_'+noise+'.npy',data)
data = np.load('result_vol_npys/gt_'+sample+'_vol.npy')
data = np.flip(data,0)
data = np.flip(data,2)
data = np.transpose(data, (2,0,1))
np.save('result_vol_npys/gt_'+sample+'flipped_vol.npy',data)

## Volume Rendering

In [ ]:
levels = {

    'UCSD900':
         {  'SR': { '10': [0.10, 0.35], '15': [0.10, 0.35], '20': [0.10, 0.35], '25': [0.10, 0.35], '30': [0.10, 0.35], '40': [0.10, 0.35]},
            'LR': { '10': [0.20, 0.45], '15': [0.20, 0.60], '20': [0.20, 0.65], '25': [0.20, 0.75], '30': [0.20, 0.80], '40': [0.20, 0.80]},
            'HR': [1.00, 0.75]
         },
    'UCSD900flipped':
         {  'SR': { '10': [0.35, 0.10], '15': [0.35, 0.10], '20': [0.35, 0.10], '25': [0.35, 0.10], '30': [0.35, 0.10], '40': [0.35, 0.10]},
            'LR': { '10': [0.35, 0.20], '15': [0.60, 0.20], '20': [0.65, 0.20], '25': [0.75, 0.20], '30': [0.80, 0.20], '40': [0.80, 0.20]},
            'HR': [1.00, 0.75]
         },
    'S1':
         {  'SR': { '10': [0.35, 0.00], '15': [0.35, 0.00], '20': [0.35, 0.00], '25': [0.35, 0.00] },
            'LR': { '10': [0.50, 0.00], '15': [0.50, 0.00], '20': [0.50, 0.00], '25': [0.50, 0.00] },
            'HR': [1.00, 0.75]
         },
    'S2':
         {  'SR': { '10': [0.35, 0.00], '15': [0.35, 0.00], '20': [0.35, 0.00], '25': [0.35, 0.00] },
            'LR': { '10': [0.50, 0.00], '15': [0.50, 0.00], '20': [0.50, 0.00], '25': [0.50, 0.00] },
            'HR': [1.00, 0.75]
         },
    'S3':
         {  'SR': { '10': [0.35, 0.00], '15': [0.35, 0.00], '20': [0.35, 0.00], '25': [0.35, 0.00] },
            'LR': { '10': [0.50, 0.00], '15': [0.50, 0.00], '20': [0.50, 0.00], '25': [0.50, 0.00] },
            'HR': [1.00, 0.75]
         }  
}

opacities = {
     'UCSD900':  
            { 'SR': { '10': [0.00,0.20,0.00], '15': [0.00,0.20,0.00], '20': [0.00,0.20,0.00], '25': [0.00,0.20,0.00]},
              'LR': { '10': [0.00,0.20,0.00], '15': [0.00,0.20,0.00], '20': [0.00,0.20,0.00], '25': [0.00,0.20,0.00]},
              'HR': [0.70,0.73,0.00]
            },
     'UCSD900flipped':  
            { 'SR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00], '30': [0.20,0.00,0.00], '40': [0.20,0.00,0.00]},
              'LR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00], '30': [0.20,0.00,0.00], '40': [0.20,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
      'S1':
            { 'SR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00]},
              'LR': { '10': [0.15,0.00,0.00], '15': [0.15,0.00,0.00], '20': [0.15,0.00,0.00], '25': [0.15,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
      'S2':
            { 'SR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00]},
              'LR': { '10': [0.15,0.00,0.00], '15': [0.15,0.00,0.00], '20': [0.15,0.00,0.00], '25': [0.15,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
       'S3':
            { 'SR': { '10': [0.20,0.00,0.00], '15': [0.20,0.00,0.00], '20': [0.20,0.00,0.00], '25': [0.20,0.00,0.00]},
              'LR': { '10': [0.15,0.00,0.00], '15': [0.15,0.00,0.00], '20': [0.15,0.00,0.00], '25': [0.15,0.00,0.00]},
              'HR': [0.70,0.73,0.00]
            },
}

In [ ]:
fig_res = {} # dictionary of result figures
fig_inp = {} # dictionary of input figures

In [ ]:
import pythreejs
figcam = pythreejs.PerspectiveCamera(aspect=0.8, fov=46.0, matrixWorldNeedsUpdate=True, position=(1.785528946985704, 1.3757523415625494, 2.412892214337929), projectionMatrix=(2.9448154572796907, 0.0, 0.0, 0.0, 0.0, 2.3558523658237527, 0.0, 0.0, 0.0, 0.0, -1.00010000500025, -1.0, 0.0, 0.0, -0.200010000500025, 0.0), quaternion=(-0.19796316984919116, 0.30892794387643674, 0.08046673488216134, 0.9267681551784899), rotation=(-0.5181948251143776, 0.5713287660544955, 0.3285944197012996, 'XYZ'), scale=(1.0, 1.0, 1.0), up=(-0.27146072492949946, 0.9086713759213286, -0.31721507751364897))

In [ ]:
#noise = '10'
noises = ['10','15','20','25']
sample = 'UCSD900flipped'
#frame_range = [200,509,0,509]
frame_range = [0,90,0,390]
opacity = 5.00
brightness = 2.00

In [ ]:
noise = '10'
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
#ipyvolume.selector_default()
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

In [ ]:
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

In [ ]:
noise = '15'
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

In [ ]:
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

In [ ]:
noise = '20'
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

In [ ]:
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

In [ ]:
noise = '25'
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam,opacity_scale = 1.58)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('output_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

In [ ]:
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

In [ ]:
noise = '30'
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

In [ ]:
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

In [ ]:
noise = '40'
avg_img = np.load('result_vol_npys/output_'+sample+'_vol_'+noise+'.npy')
avg_img = avg_img[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = avg_img.min(axis=(0,1))
maxs = avg_img.max(axis=(0,1))
avg_img_norm = (avg_img - mins) / (maxs - mins)
fig_res[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(avg_img_norm[:,:,:],level=levels[sample]['SR'][noise],opacity=opacities[sample]['SR'][noise])
fig_res[noise].volumes[0].opacity_scale = opacity
fig_res[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_res[noise])

In [ ]:
input_vol = np.load('result_vol_npys/input_'+sample+'_vol_'+noise+'.npy')
input_vol = input_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = input_vol.min(axis=(0,1))
maxs = input_vol.max(axis=(0,1))
input_vol_norm = (input_vol - mins) / (maxs - mins)
fig_inp[noise] = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(input_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise])
fig_inp[noise].volumes[0].opacity_scale = opacity
fig_inp[noise].volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('input_3dfig_'+sample+'_'+noise+'.png',width=1280,height=720,fig=fig_inp[noise])

In [ ]:
gt_vol = np.load('result_vol_npys/gt_'+sample+'_vol.npy')
gt_vol = gt_vol[:,frame_range[0]:frame_range[1],frame_range[2]:frame_range[3]]
mins = gt_vol.min(axis=(0,1))
maxs = gt_vol.max(axis=(0,1))
gt_vol_norm = (gt_vol - mins) / (maxs - mins)
fig_gt = ipyvolume.figure(camera = figcam)
ipyvolume.volshow(gt_vol_norm[:,:,:],level=levels[sample]['LR'][noise],opacity=opacities[sample]['LR'][noise],specular_exponent=5)
fig_gt.volumes[0].opacity_scale = opacity
fig_gt.volumes[0].brightness = brightness
ipyvolume.pylab.xyzlabel(' ', ' ', ' ')
ipyvolume.show()

In [ ]:
ipyvolume.pylab.savefig('gt_3dfig_'+sample+'.png',width=1280,height=720,fig = fig_gt)

## Convert full UCSD sample data into MATLAB

In [ ]:
for noise in noises:
    sr_data = np.load('result_vol_npys/output_ucsd_vol_'+noise+'.npy')
    lr_data = np.load('result_vol_npys/input_ucsd_vol_'+noise+'.npy')
    mins = sr_data.min(axis=(0,1))
    maxs = sr_data.max(axis=(0,1))
    sr_data_norm = (sr_data - mins) / (maxs - mins)
    mins = lr_data.min(axis=(0,1))
    maxs = lr_data.max(axis=(0,1))
    lr_data_norm = (lr_data - mins) / (maxs - mins)    
    sio.savemat('UCSD900_'+noise+'_LR.mat',{'vect':lr_data})
    sio.savemat('UCSD900_'+noise+'_SR.mat',{'vect':sr_data})
    sio.savemat('UCSD900_norm_'+noise+'_LR.mat',{'vect':lr_data_norm})
    sio.savemat('UCSD900_norm_'+noise+'_SR.mat',{'vect':sr_data_norm})
hr_data = np.load('target_ucsd_vol.npy')
mins = hr_data.min(axis=(0,1))
maxs = hr_data.max(axis=(0,1))
hr_data_norm = (hr_data - mins) / (maxs - mins) 
sio.savemat('UCSD900_HR.mat',{'vect':hr_data})
sio.savemat('UCSD900_norm_HR.mat',{'vect':hr_data_norm})